In [31]:
!pip install pandas
!pip install numpy

In [32]:
import pandas as pd
import numpy as np


In [42]:
data_file_path = 'welddb/welddb.data'
headers_file_path = 'welddb/headers.txt'
with open(headers_file_path, 'r') as f:
    headers = [line.strip() for line in f]
df = pd.read_csv(data_file_path, sep=r'\s+', header=None, names=headers)

In [43]:
df

,Carbon concentration (weight%),Silicon concentration (weight%),Manganese concentration (weight%),Sulphur concentration (weight%),Phosphorus concentration (weight%),Nickel concentration (weight%),Chromium concentration (weight%),Molybdenum concentration (weight%),Vanadium concentration (weight%),Copper concentration (weight%),...,Charpy temperature (Â°C),Charpy impact toughness (J),Hardness (kg/mmÂ²),50% FATT,Primary ferrite (%),Ferrite with second phase (%),Acicular ferrite (%),Martensite (%),Ferrite with carbide aggregate (%),Weld ID
0,0.037,0.30,0.65,0.008,0.012,0,N,N,N,N,...,N,N,N,N,N,N,N,N,N,Evans-Ni/CMn-1990/1991-0Aaw
1,0.037,0.30,0.65,0.008,0.012,0,N,N,N,N,...,-28,100,N,N,N,N,N,N,N,Evans-Ni/CMn-1990/1991-0Aawch
2,0.037,0.30,0.65,0.008,0.012,0,N,N,N,N,...,-38,100,N,N,N,N,N,N,N,Evans-Ni/CMn-1990/1991-0Aht
3,0.037,0.31,1.03,0.007,0.014,0,N,N,N,N,...,N,N,N,N,N,N,N,N,N,Evans-Ni/CMn-1990/1991-0Baw
4,0.037,0.31,1.03,0.007,0.014,0,N,N,N,N,...,-48,100,N,N,32,28,40,0,0,Evans-Ni/CMn-1990/1991-0Bawch
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1647,0.100,0.35,0.90,0.008,0.016,0.60,8.6,0.98,0.18,N,...,N,N,N,N,N,N,N,N,N,Birmingham-MAX35
1648,0.088,0.36,0.88,0.008,0.017,0.57,8.4,0.94,0.19,N,...,N,N,N,N,N,N,N,N,N,Birmingham-MAX36
1649,0.090,0.34,0.89,0.008,0.016,0.17,8.2,0.94,0.02,N,...,N,N,N,N,N,N,N,N,N,Birmingham-MAX37
1650,0.092,0.35,0.90,0.008,0.016,0.54,8.4,0.97,0.17,N,...,N,N,N,N,N,N,N,N,N,Birmingham-MAX38


Let's see the vartiables types so that we can convert the types of the variables to the right format

In [44]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1652 entries, 0 to 1651
Data columns (total 44 columns):
 #   Column                                      Non-Null Count  Dtype  
---  ------                                      --------------  -----  
 0   Carbon concentration (weight%)              1652 non-null   float64
 1   Silicon concentration (weight%)             1652 non-null   float64
 2   Manganese concentration (weight%)           1652 non-null   float64
 3   Sulphur concentration (weight%)             1652 non-null   object 
 4   Phosphorus concentration (weight%)          1652 non-null   object 
 5   Nickel concentration (weight%)              1652 non-null   object 
 6   Chromium concentration (weight%)            1652 non-null   object 
 7   Molybdenum concentration (weight%)          1652 non-null   object 
 8   Vanadium concentration (weight%)            1652 non-null   object 
 9   Copper concentration (weight%)              1652 non-null   object 
 10  Cobalt conce

In [45]:
#Replace missing values with NaN
df.replace('N', np.nan, inplace=True)

We should convert all te numerical variables to numerical type

In [38]:
#see the 1197th row
df.loc[1197]

Carbon concentration (weight%)                           0.06
Silicon concentration (weight%)                          0.32
Manganese concentration (weight%)                        1.23
Sulphur concentration (weight%)                        <0.002
Phosphorus concentration (weight%)                      0.002
Nickel concentration (weight%)                           1.01
Chromium concentration (weight%)                          8.4
Molybdenum concentration (weight%)                       0.89
Vanadium concentration (weight%)                         0.22
Copper concentration (weight%)                          <0.01
Cobalt concentration (weight%)                          <0.01
Tungsten concentration (weight%)                          NaN
Oxygen concentration (ppm)                                380
Titanium concentration (ppm)                             <100
Nitrogen concentration (ppm)                               80
Aluminium concentration (ppm)                            <100
Boron co

Some lines that are expected to have numerical values may contain some non numerical one (like the '<0.002' value assigned to the variable 'Sulphur concentration (weight%)' for the 1197th row of the dataframe)

First, we will convert only possible rows then we will treat the rest

In [47]:
#Convert the variables to numeric if possible
df_automatic_numeric = df.apply(pd.to_numeric, errors='ignore')

C:\Users\moham\AppData\Local\Temp\ipykernel_9932\1888536335.py:2: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df_automatic_numeric = df.apply(pd.to_numeric, errors='ignore')


In [48]:
df_automatic_numeric.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1652 entries, 0 to 1651
Data columns (total 44 columns):
 #   Column                                      Non-Null Count  Dtype  
---  ------                                      --------------  -----  
 0   Carbon concentration (weight%)              1652 non-null   float64
 1   Silicon concentration (weight%)             1652 non-null   float64
 2   Manganese concentration (weight%)           1652 non-null   float64
 3   Sulphur concentration (weight%)             1648 non-null   object 
 4   Phosphorus concentration (weight%)          1642 non-null   float64
 5   Nickel concentration (weight%)              697 non-null    float64
 6   Chromium concentration (weight%)            784 non-null    float64
 7   Molybdenum concentration (weight%)          793 non-null    object 
 8   Vanadium concentration (weight%)            928 non-null    object 
 9   Copper concentration (weight%)              578 non-null    object 
 10  Cobalt conce

We can see that already 23 columns have no problems.
Now, we will deal with the other numerical columns that may contain some anomalies (like the '<' in the last example)